In [15]:
import sys
sys.path.append('../')

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import sqlite3
import akshare as ak
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm

from database_auto.db_data_downloader.downloader_base import DownloaderBase
import database_auto.database_config as db_config

pd.options.display.max_rows=None
pd.options.display.max_columns=None

!python --version

Python 3.8.10


In [16]:
import tensorflow as tf

# 只使用CPU进行训练
tf.config.set_visible_devices([], 'GPU')

# 打印Tensorflow版本
print(f"Tensorflow Version: {tf.__version__}")

# 检查是否有可用的GPU设备
if tf.test.is_built_with_cuda():
    print("TensorFlow GPU version is installed")
else:
    print("TensorFlow CPU version is installed")

# 检查TensorFlow是否能够访问GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPU devices available:", gpus)
else:
    print("No GPU devices found. Running on CPU.")

# !nvidia-smi

Tensorflow Version: 2.13.1
TensorFlow GPU version is installed
GPU devices available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [17]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# 绘图相关函数
def plot_series_dist(series):
    data = series
    plt.figure(figsize=(5,5))
    # 使用matplotlib画直方图
    plt.hist(data, bins=60, edgecolor='k', alpha=0.7)
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.title('Histogram of Data')
    plt.show()

def plot_metrics(history):
  metrics = ['loss', 'mean_squared_error']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    plt.ylim([0, plt.ylim()[1]])
    plt.legend()

def plot_cm(true_labels, pred_labels):
    cm = confusion_matrix(true_labels, pred_labels)
    plt.figure(figsize=(5,5))
    sns.heatmap(cm, annot=True, fmt="g", cmap='Blues')
    plt.title('Confusion matrix')
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')

    accuracy = accuracy_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels)
    recall = recall_score(true_labels, pred_labels)
    print(f"Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}")

In [18]:
class PreProcessing:
    def __init__(self, db_downloader:DownloaderBase) -> None:
        self.db_downloader = db_downloader

    def _build_reg_label(self, stock_dataframe, N=15):
        stock_df = stock_dataframe.copy()
        # 计算未来N日内的最高收益率
        stock_df['max_return'] = stock_df['close'].rolling(window=N).max().shift(-N) / stock_df['close'] - 1
        # 计算未来N日内的最低收益率
        stock_df['min_return'] = stock_df['close'].rolling(window=N).min().shift(-N) / stock_df['close'] - 1
        # 计算未来N日内的收益率和（期望最高收益率越高越好，最低收益率也越高越好，由于最低收益率是负数，因此使用最高+最低来作为综合收益指标）
        stock_df['return_sum'] = stock_df['max_return'] + stock_df['min_return']
        stock_df['label'] = stock_df['return_sum']
        # 过滤第二天一字涨停股票
        stock_df = stock_df[stock_df['high'].shift(-1) != stock_df['low'].shift(-1)]
        return stock_df[['datetime', 'label']]

    def _process_one_stock(self, stock_code, start_date, end_date):
        stock_history = self.db_downloader._download_history_base_info(stock_code, start_date, end_date)
        stock_profile = self.db_downloader._download_all_stock_info(stock_code)
        stock_indicator = self.db_downloader._download_history_indicator_info(stock_code, start_date, end_date)
        stock_indicator = stock_indicator.replace("", np.NaN).ffill()
        stock_factor_date = self.db_downloader._download_history_date_factor_info(start_date, end_date)
        stock_factor_alpha184 = self.db_downloader._download_history_alpha184_factor_info_old_version(stock_code, start_date, end_date)
        stock_label = self._build_reg_label(stock_history)

        stock_df = stock_history.merge(stock_profile, on=['code']) \
                .merge(stock_indicator, on=['code', 'datetime']) \
                .merge(stock_factor_alpha184, on=['code', 'datetime']) \
                .merge(stock_factor_date, on=['datetime']) \
                .merge(stock_label, on=['datetime'])
        stock_df = stock_df.dropna()
        return stock_df

    # def _get_index_hist_cons(self, benchmark, start_date, end_date):
    #     import baostock as bs
    #     bs.login()
    #     index_stock_cons_set = set()
    #     for cur_date in tqdm(pd.date_range(start=start_date, end=end_date, freq='B')):
    #         cur_date = datetime.strftime(cur_date, '%Y-%m-%d')
    #         if benchmark == '000016':
    #             # print("开始处理上证50...")
    #             dataframe = bs.query_sz50_stocks(date=cur_date).get_data()
    #         elif benchmark == '000300':
    #             # print("开始处理沪深300...")
    #             dataframe = bs.query_hs300_stocks(date=cur_date).get_data()
    #         elif benchmark == '000905':
    #             # print("开始处理中证500...")
    #             dataframe = bs.query_zz500_stocks(date=cur_date).get_data()
    #         if not dataframe.empty:
    #             index_stock_cons_set.update(
    #                 dataframe['code'].unique()
    #             )
    #     bs.logout()
    #     return index_stock_cons_set
    
    def _get_index_latest_cons(self, benchmark):
        import baostock as bs
        bs.login()
        if benchmark == '000016':
            dataframe = bs.query_sz50_stocks().get_data()
        elif benchmark == '000300':
            dataframe = bs.query_hs300_stocks().get_data()
        return dataframe['code'].unique()

    def _process_all_stock(self, benchmark, start_date, end_date):
        # 获取区间内benchmark的所有成份股
        # stock_code_list = self._get_index_hist_cons(benchmark, start_date, end_date)
        stock_code_list = self._get_index_latest_cons(benchmark)
        stock_df_list = []
        for stock_code in tqdm(stock_code_list, desc=f'Process: {benchmark} ...'):
            stock_df = self._process_one_stock(stock_code, start_date, end_date)
            if not stock_df.empty:
                stock_df_list.append(stock_df)
        return pd.concat(stock_df_list)

In [19]:
def get_rolling_data_period(backtest_start_date, backtest_duration=5, train_period=6, val_period=0.5, test_period=0.5):
    """
    Args:
        backtest_start_date (_type_): _description_
        backtest_duration (int, optional): _description_. Defaults to 5.
        train_period (int, optional): _description_. Defaults to 6.
        val_period (float, optional): _description_. Defaults to 0.5.
        test_period (float, optional): _description_. Defaults to 0.5.
    Returns:
        result: _description_
    """
    backtest_start_date = datetime.strptime(backtest_start_date, '%Y%m%d')
    backtest_end_date = backtest_start_date + relativedelta(years=backtest_duration) # 回测5年数据
    train_period = relativedelta(years=train_period) # 使用6年的训练数据
    val_period = relativedelta(months=(12 * val_period)) # 使用半年的验证数据
    test_period = relativedelta(months=(12 * test_period)) # 使用半年的测试数据(半年模型一更新)

    result = []
    rolling_flag = True
    bench_date = backtest_start_date
    while rolling_flag:
        if bench_date < backtest_end_date:
            test_start, test_end = bench_date, (bench_date + test_period - relativedelta(days=1))
            val_start, val_end = (test_start - relativedelta(days=1) - val_period), (test_start - relativedelta(days=1))
            train_start, train_end =(val_start - relativedelta(days=1) - train_period), (val_start - relativedelta(days=1))
            result.append({
                "train": [train_start.strftime("%Y%m%d"), train_end.strftime("%Y%m%d")],
                "val": [val_start.strftime("%Y%m%d"), val_end.strftime("%Y%m%d")],
                "test": [test_start.strftime("%Y%m%d"), test_end.strftime("%Y%m%d")]
            })
            bench_date += test_period
        else:
            rolling_flag = False 
    return result

In [20]:
def extract_train_val_data(df, train_start_date, train_end_date, val_start_date, val_end_date, test_start_date, test_end_date):
    train_start_date = pd.to_datetime(train_start_date)
    train_end_date = pd.to_datetime(train_end_date)
    val_start_date = pd.to_datetime(val_start_date)
    val_end_date = pd.to_datetime(val_end_date)
    test_start_date = pd.to_datetime(test_start_date)
    test_end_date = pd.to_datetime(test_end_date)

    train_data = df[(pd.to_datetime(df['datetime']) >= train_start_date) & (pd.to_datetime(df['datetime']) <= train_end_date)]
    val_data = df[(pd.to_datetime(df['datetime']) >= val_start_date) & (pd.to_datetime(df['datetime']) <= val_end_date)]
    test_data = df[(pd.to_datetime(df['datetime']) >= test_start_date) & (pd.to_datetime(df['datetime']) <= test_end_date)]

    print(f"train_data_size: {train_data.shape}")
    print(f"validation_data_size: {val_data.shape}")
    print(f"test_data_size: {test_data.shape}")
    return train_data, val_data, test_data

def df_to_dataset(dataframe, feature_cols, label_cols, shuffle=False, batch_size=32):
    features = dataframe[feature_cols]
    labels = dataframe[label_cols]
    ds = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=min(len(features), 10000), seed=1024)
    ds = ds.cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer


class QuantileClipTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, lower_quantile=0.01, upper_quantile=0.99):
        self.lower_quantile = lower_quantile
        self.upper_quantile = upper_quantile
        
    def fit(self, X, y=None):
        # 计算给定分位数的分界值
        self.lower_bound_ = np.nanquantile(X, self.lower_quantile, axis=0)
        self.upper_bound_ = np.nanquantile(X, self.upper_quantile, axis=0)
        return self
    
    def transform(self, X, y=None):
        # 对整个数组应用剪辑操作
        return np.clip(X, self.lower_bound_, self.upper_bound_)

    def fit_transform(self, X, y=None, **fit_params):
        return self.fit(X, y).transform(X)

In [22]:
# 数据库初始化
db_conn = sqlite3.connect('../database_auto/hh_quant_auto.db')
db_downloader = DownloaderBase(db_conn, db_config)
proprocessor = PreProcessing(db_downloader=db_downloader)

# 相关配置
rolling_flag = False
# benchmark = '000905'
benchmark = '000016'  # 上证50
# benchmark = '000300' # 沪深300

feature_config = {
    "target_features": ['label'],
    "numeric_features": ['turnover_rate', 'pe_ttm', 'ps_ttm', 'pcf_ncf_ttm', 'pb_mrq', 'KMID', 'KLEN', 'KMID2', 'KUP', 'KUP2', 'KLOW', 'KLOW2', 'KSFT', 'KSFT2', 'OPEN0', 'OPEN1', 'OPEN2', 'OPEN3', 'OPEN4', 'HIGH0', 'HIGH1', 'HIGH2', 'HIGH3', 'HIGH4', 'LOW0', 'LOW1', 'LOW2', 'LOW3', 'LOW4', 'CLOSE1', 'CLOSE2', 'CLOSE3', 'CLOSE4', 'VOLUME1', 'VOLUME2', 'VOLUME3', 'VOLUME4', 'ROC5', 'ROC10', 'ROC20', 'ROC30', 'ROC60', 'MAX5', 'MAX10', 'MAX20', 'MAX30', 'MAX60', 'MIN5', 'MIN10', 'MIN20', 'MIN30', 'MIN60', 'MA5', 'MA10', 'MA20', 'MA30', 'MA60', 'STD5', 'STD10', 'STD20', 'STD30', 'STD60', 'BETA5', 'BETA10', 'BETA20', 'BETA30', 'BETA60', 'RSQR5', 'RSQR10', 'RSQR20', 'RSQR30', 'RSQR60', 'RESI5', 'RESI10', 'RESI20', 'RESI30', 'RESI60', 'QTLU5', 'QTLU10', 'QTLU20', 'QTLU30', 'QTLU60', 'QTLD5', 'QTLD10', 'QTLD20', 'QTLD30', 'QTLD60', 'TSRANK5', 'TSRANK10', 'TSRANK20', 'TSRANK30', 'TSRANK60', 'RSV5', 'RSV10', 'RSV20', 'RSV30', 'RSV60', 'IMAX5', 'IMAX10', 'IMAX20', 'IMAX30', 'IMAX60', 'IMIN5', 'IMIN10', 'IMIN20', 'IMIN30', 'IMIN60', 'IMXD5', 'IMXD10', 'IMXD20', 'IMXD30', 'IMXD60', 'CORR5', 'CORR10', 'CORR20', 'CORR30', 'CORR60', 'CORD5', 'CORD10', 'CORD20', 'CORD30', 'CORD60', 'CNTP5', 'CNTP10', 'CNTP20', 'CNTP30', 'CNTP60', 'CNTN5', 'CNTN10', 'CNTN20', 'CNTN30', 'CNTN60', 'CNTD5', 'CNTD10', 'CNTD20', 'CNTD30', 'CNTD60', 'SUMP5', 'SUMP10', 'SUMP20', 'SUMP30', 'SUMP60', 'SUMN5', 'SUMN10', 'SUMN20', 'SUMN30', 'SUMN60', 'SUMD5', 'SUMD10', 'SUMD20', 'SUMD30', 'SUMD60', 'VMA5', 'VMA10', 'VMA20', 'VMA30', 'VMA60', 'VSTD5', 'VSTD10', 'VSTD20', 'VSTD30', 'VSTD60', 'WVMA5', 'WVMA10', 'WVMA20', 'WVMA30', 'WVMA60', 'VSUMP5', 'VSUMP10', 'VSUMP20', 'VSUMP30', 'VSUMP60', 'VSUMN5', 'VSUMN10', 'VSUMN20', 'VSUMN30', 'VSUMN60', 'VSUMD5', 'VSUMD10', 'VSUMD20', 'VSUMD30', 'VSUMD60'],
    "integer_categorical_features": ['month'],
    "string_categorical_features": ['industry', 'season'],
}
batch_size = 1024

# 是否开启滚动训练&回测
if rolling_flag:
    print("开启滚动回测...")
    backtest_period = get_rolling_data_period(
        backtest_start_date='20200101', # 回测开始日期
        backtest_duration=4, # 一共回测多久的数据（单位：年）
        train_period=6, # 使用过去多久的时间进行训练（单位：年）
        val_period=1, # 验证数据周期（单位：年）
        test_period=1, # 测试数据周期（单位：年）
    )
else:
    print("关闭滚动回测...")
    backtest_period = [
        {
            'train': ['2009-01-01', '2016-12-31'], # 8年训练
            'val': ['2017-01-01', '2018-12-31'], # 2年验证
            'test': ['2019-01-01', '2024-12-31'] # 5年+预测
        }
    ]

backtest_period

关闭滚动回测...


[{'train': ['2009-01-01', '2016-12-31'],
  'val': ['2017-01-01', '2018-12-31'],
  'test': ['2019-01-01', '2024-12-31']}]

In [23]:
df = proprocessor._process_all_stock('000016', '2009-01-01', '2024-12-31')
# df[(df['code'] == 'sh.688981') & (df['datetime'] == '2024-04-17')].T

login success!


Process: 000016 ...: 100%|██████████| 50/50 [00:09<00:00,  5.50it/s]


In [25]:
date_period_params = backtest_period[0]
print(date_period_params)
train_start_date, train_end_date = date_period_params['train']
val_start_date, val_end_date = date_period_params['val']
test_start_date, test_end_date = date_period_params['test']
# 获取全区间数据
print("开始加载原始数据...")
df = proprocessor._process_all_stock(benchmark=benchmark, start_date=train_start_date, end_date=test_end_date)
# 抽取训练验证数据
print("开始拆分训练、验证、测试集合...")
train_data, val_data, test_data = extract_train_val_data(df, *[train_start_date, train_end_date, val_start_date, val_end_date, test_start_date, test_end_date])
# 从data中抽取相关特征数据
print("开始抽取特征数据...")
feature_columns = feature_config.get('numeric_features', []) + feature_config.get('integer_categorical_features', []) + feature_config.get('string_categorical_features', [])
label_columns = feature_config.get('target_features', [])
full_feature_columns = feature_columns + label_columns
train_df, val_df, test_df = train_data[full_feature_columns], val_data[full_feature_columns], test_data[full_feature_columns]
# 对相关特征进行特征工程
print("开始对特征进行预处理...")
feature_preprocess_pipeline = Pipeline(steps=[
    ('robust_scaler', RobustScaler()),
    # ('quantile_transformer', QuantileTransformer(n_quantiles=5000, output_distribution='uniform', random_state=42)),
    ('minmax_scaler', MinMaxScaler()),
])
preprocess_feature_columns = feature_config.get('numeric_features', [])
train_df[preprocess_feature_columns] = feature_preprocess_pipeline.fit_transform(train_df[preprocess_feature_columns])
val_df[preprocess_feature_columns] = feature_preprocess_pipeline.transform(val_df[preprocess_feature_columns])
test_df[preprocess_feature_columns] = feature_preprocess_pipeline.transform(test_df[preprocess_feature_columns])

print("开始对标签进行预处理...")
label_preprocess_pipeline = Pipeline(steps=[
    ('quantile_clipper', QuantileClipTransformer()),
    ('robust_scaler', RobustScaler()),
    # ('quantile_transformer', QuantileTransformer(n_quantiles=1000, output_distribution='normal', random_state=42)),
    ('minmax_scaler', MinMaxScaler()),
])
preprocess_target_columns = feature_config.get('target_features', [])
train_df[preprocess_target_columns] = label_preprocess_pipeline.fit_transform(train_df[preprocess_target_columns])
val_df[preprocess_target_columns] = label_preprocess_pipeline.transform(val_df[preprocess_target_columns])
test_df[preprocess_target_columns] = label_preprocess_pipeline.transform(test_df[preprocess_target_columns])

{'train': ['2009-01-01', '2016-12-31'], 'val': ['2017-01-01', '2018-12-31'], 'test': ['2019-01-01', '2024-12-31']}
开始加载原始数据...
login success!


Process: 000016 ...: 100%|██████████| 50/50 [00:08<00:00,  5.60it/s]


开始拆分训练、验证、测试集合...
train_data_size: (50924, 204)
validation_data_size: (18461, 204)
test_data_size: (59602, 204)
开始抽取特征数据...
开始对特征进行预处理...
开始对标签进行预处理...


In [26]:
train_df.head()

,turnover_rate,pe_ttm,ps_ttm,pcf_ncf_ttm,pb_mrq,KMID,KLEN,KMID2,KUP,KUP2,KLOW,KLOW2,KSFT,KSFT2,OPEN0,OPEN1,OPEN2,OPEN3,OPEN4,HIGH0,HIGH1,HIGH2,HIGH3,HIGH4,LOW0,LOW1,LOW2,LOW3,LOW4,CLOSE1,CLOSE2,CLOSE3,CLOSE4,VOLUME1,VOLUME2,VOLUME3,VOLUME4,ROC5,ROC10,ROC20,ROC30,ROC60,MAX5,MAX10,MAX20,MAX30,MAX60,MIN5,MIN10,MIN20,MIN30,MIN60,MA5,MA10,MA20,MA30,MA60,STD5,STD10,STD20,STD30,STD60,BETA5,BETA10,BETA20,BETA30,BETA60,RSQR5,RSQR10,RSQR20,RSQR30,RSQR60,RESI5,RESI10,RESI20,RESI30,RESI60,QTLU5,QTLU10,QTLU20,QTLU30,QTLU60,QTLD5,QTLD10,QTLD20,QTLD30,QTLD60,TSRANK5,TSRANK10,TSRANK20,TSRANK30,TSRANK60,RSV5,RSV10,RSV20,RSV30,RSV60,IMAX5,IMAX10,IMAX20,IMAX30,IMAX60,IMIN5,IMIN10,IMIN20,IMIN30,IMIN60,IMXD5,IMXD10,IMXD20,IMXD30,IMXD60,CORR5,CORR10,CORR20,CORR30,CORR60,CORD5,CORD10,CORD20,CORD30,CORD60,CNTP5,CNTP10,CNTP20,CNTP30,CNTP60,CNTN5,CNTN10,CNTN20,CNTN30,CNTN60,CNTD5,CNTD10,CNTD20,CNTD30,CNTD60,SUMP5,SUMP10,SUMP20,SUMP30,SUMP60,SUMN5,SUMN10,SUMN20,SUMN30,SUMN60,SUMD5,SUMD10,SUMD20,SUMD30,SUMD60,VMA5,VMA10,VMA20,VMA30,VMA60,VSTD5,VSTD10,VSTD20,VSTD30,VSTD60,WVMA5,WVMA10,WVMA20,WVMA30,WVMA60,VSUMP5,VSUMP10,VSUMP20,VSUMP30,VSUMP60,VSUMN5,VSUMN10,VSUMN20,VSUMN30,VSUMN60,VSUMD5,VSUMD10,VSUMD20,VSUMD30,VSUMD60,month,industry,season,label
0,0.005568,0.841578,0.012192,0.877479,0.041680,0.482108,0.095335,0.800172,0.025570,0.133028,0.046211,0.266628,0.489105,8.669725e-01,0.418434,0.387175,0.373768,0.358275,0.325918,0.012490,0.212298,0.262395,0.303919,0.260756,0.900455,0.605394,0.542480,0.397019,0.359173,0.335271,0.384364,0.380911,0.353711,0.080716,0.084997,0.117099,0.050809,0.317632,0.329123,0.379469,0.322539,0.489385,0.002645,0.063316,0.193464,0.229166,0.261840,0.597133,0.981827,0.981740,0.981651,0.981381,0.365780,0.336235,0.340839,0.333201,0.388849,0.042353,0.098343,0.159695,0.146417,0.207034,0.636301,0.648305,0.570547,0.577404,0.521416,0.236434,0.622743,0.915534,0.607031,0.100373,0.558938,0.508894,0.602363,0.555440,0.739108,0.133720,0.176910,0.244363,0.261329,0.333339,0.688320,0.650492,0.500004,0.406848,0.434894,1.000,0.555556,0.263158,0.172414,0.237288,0.993142,0.084408,0.012823,0.006939,0.002919,0.50,1.000000,0.947368,0.620690,0.966102,0.75,0.333333,0.157895,0.103448,0.711864,0.375,0.833333,0.894737,0.758621,0.627119,0.459666,0.945243,0.874062,0.804459,0.597788,0.782145,0.641823,0.686787,0.730910,0.674873,0.4,0.3,0.3750,0.45,0.482759,0.6,0.7,0.666667,0.6,0.642857,0.4,0.30,0.354839,0.416667,0.423077,0.647103,0.191657,0.191953,0.313155,0.279049,0.352897,0.808343,0.808047,0.686845,0.720951,0.647103,0.191657,0.191953,0.313155,0.279049,0.089666,0.130229,0.143430,0.117604,0.145318,0.015632,0.120173,0.191937,0.139051,0.074666,0.317027,0.373854,0.230806,0.190433,0.114214,0.291000,0.477762,0.402884,0.440317,0.481866,0.709000,0.522238,0.597116,0.559683,0.518134,0.291000,0.477762,0.402884,0.440317,0.481866,1,化工,Winter,0.542538
1,0.011445,0.841601,0.012667,0.877592,0.043481,0.540810,0.201930,0.906317,0.038087,0.093548,0.034441,0.093817,0.540840,9.064516e-01,0.362528,0.348473,0.331699,0.339676,0.282332,0.018179,0.142945,0.218419,0.271066,0.242634,0.784590,0.567675,0.465807,0.395165,0.310664,0.305667,0.296482,0.327165,0.327114,0.038346,0.030523,0.054908,0.028203,0.284172,0.312786,0.357217,0.308196,0.448410,0.003849,0.051854,0.183595,0.219733,0.252808,0.579334,0.960675,0.960486,0.960295,0.959710,0.318609,0.293998,0.310850,0.309907,0.364895,0.098950,0.063478,0.143334,0.140919,0.189089,0.670610,0.686082,0.586621,0.581267,0.531282,0.578460,0.157109,0.835817,0.596974,0.075304,0.589188,0.519562,0.633186,0.593862,0.757134,0.104862,0.141381,0.205611,0.234259,0.314904,0.611737,0.605687,0.471065,0.385084,0.415047,1.000,0.888889,0.421053,0.275862,0.338983,0.990467,0.195876,0.028768,0.015524,0.006520,0.00,1.000000,1.000000,0.655172,0.983051,1.00,0.444444,0.210526,0.137931,0.728814,0.000,0.777778,0.894737,0.758621,0.627119,0.901394,0.908289,0.879009,0.794301,0.617869,0.882316,0.780526,0.748364,0.762580,0.690030,0.6,0.3,0.4375,0.50,0.517241,0.4,0.7,0.600000,0.6,0.607143,0

In [27]:
# feature_config = {
#     "target_features": ['label'],
#     "numeric_features": ['turnover_rate', 'pe_ttm', 'ps_ttm', 'pcf_ncf_ttm', 'pb_mrq', 'KMID', 'KLEN', 'KMID2', 'KUP', 'KUP2', 'KLOW', 'KLOW2', 'KSFT', 'KSFT2', 'OPEN0', 'OPEN1', 'OPEN2', 'OPEN3', 'OPEN4', 'HIGH0', 'HIGH1', 'HIGH2', 'HIGH3', 'HIGH4', 'LOW0', 'LOW1', 'LOW2', 'LOW3', 'LOW4', 'CLOSE1', 'CLOSE2', 'CLOSE3', 'CLOSE4', 'VOLUME1', 'VOLUME2', 'VOLUME3', 'VOLUME4', 'ROC5', 'ROC10', 'ROC20', 'ROC30', 'ROC60', 'MAX5', 'MAX10', 'MAX20', 'MAX30', 'MAX60', 'MIN5', 'MIN10', 'MIN20', 'MIN30', 'MIN60', 'MA5', 'MA10', 'MA20', 'MA30', 'MA60', 'STD5', 'STD10', 'STD20', 'STD30', 'STD60', 'BETA5', 'BETA10', 'BETA20', 'BETA30', 'BETA60', 'RSQR5', 'RSQR10', 'RSQR20', 'RSQR30', 'RSQR60', 'RESI5', 'RESI10', 'RESI20', 'RESI30', 'RESI60', 'QTLU5', 'QTLU10', 'QTLU20', 'QTLU30', 'QTLU60', 'QTLD5', 'QTLD10', 'QTLD20', 'QTLD30', 'QTLD60', 'TSRANK5', 'TSRANK10', 'TSRANK20', 'TSRANK30', 'TSRANK60', 'RSV5', 'RSV10', 'RSV20', 'RSV30', 'RSV60', 'IMAX5', 'IMAX10', 'IMAX20', 'IMAX30', 'IMAX60', 'IMIN5', 'IMIN10', 'IMIN20', 'IMIN30', 'IMIN60', 'IMXD5', 'IMXD10', 'IMXD20', 'IMXD30', 'IMXD60', 'CORR5', 'CORR10', 'CORR20', 'CORR30', 'CORR60', 'CORD5', 'CORD10', 'CORD20', 'CORD30', 'CORD60', 'CNTP5', 'CNTP10', 'CNTP20', 'CNTP30', 'CNTP60', 'CNTN5', 'CNTN10', 'CNTN20', 'CNTN30', 'CNTN60', 'CNTD5', 'CNTD10', 'CNTD20', 'CNTD30', 'CNTD60', 'SUMP5', 'SUMP10', 'SUMP20', 'SUMP30', 'SUMP60', 'SUMN5', 'SUMN10', 'SUMN20', 'SUMN30', 'SUMN60', 'SUMD5', 'SUMD10', 'SUMD20', 'SUMD30', 'SUMD60', 'VMA5', 'VMA10', 'VMA20', 'VMA30', 'VMA60', 'VSTD5', 'VSTD10', 'VSTD20', 'VSTD30', 'VSTD60', 'WVMA5', 'WVMA10', 'WVMA20', 'WVMA30', 'WVMA60', 'VSUMP5', 'VSUMP10', 'VSUMP20', 'VSUMP30', 'VSUMP60', 'VSUMN5', 'VSUMN10', 'VSUMN20', 'VSUMN30', 'VSUMN60', 'VSUMD5', 'VSUMD10', 'VSUMD20', 'VSUMD30', 'VSUMD60'],
#     "integer_categorical_features": ['month'],
#     "string_categorical_features": ['industry', 'season'],
# }
plot_series_dist(train_df['label'])

In [28]:
# 转换为tensorflow所使用的dataset
print("开始将DataFrame转换为DataSet...")
train_ds = df_to_dataset(train_df, feature_columns, label_columns, shuffle=True, batch_size=batch_size)
val_ds = df_to_dataset(val_df, feature_columns, label_columns, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test_df, feature_columns, label_columns, shuffle=False, batch_size=batch_size)  

开始将DataFrame转换为DataSet...


In [29]:
for feas, label in train_ds.take(1):
    print(label)

tf.Tensor(
[[0.3615577 ]
 [0.12815645]
 [0.25016917]
 ...
 [0.19307439]
 [0.61549733]
 [0.39409232]], shape=(1024, 1), dtype=float64)


2024-06-12 02:44:19.827356: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [13]:
# 准备模型训练
print("开始模型初始化 & 训练...")
from models.single_task.model_moe import QuantModel
model_config = {
        "seed": 1024,
        "feature_use_embedding": False,
        "feature_embedding_dims": 4,
        "numeric_features_with_boundaries": {k: pd.qcut(train_df[k], q=20, retbins=True, duplicates='drop')[1].tolist() for k in feature_config.get('numeric_features', [])},
        "integer_categorical_features_with_vocab": {k: list(train_df[k].unique()) for k in feature_config.get('integer_categorical_features', [])},
        "string_categorical_features_with_vocab": {k: list(train_df[k].unique()) for k in feature_config.get('string_categorical_features', [])},
    }
model = QuantModel(config=model_config)

initial_learning_rate = 5e-4

# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate,
#     decay_steps=5 * (len(train_df) // batch_size), # 每5个batch进行一次调整
#     decay_rate=0.9,
#     staircase=True)
model.compile(
    # optimizer=tf.keras.optimizers.Adam(lr_schedule),
    optimizer=tf.keras.optimizers.Adam(initial_learning_rate),
    # loss=tf.keras.losses.MeanSquaredError(),
    # loss=tf.keras.losses.MeanAbsoluteError(),
    loss=tf.keras.losses.Huber(),
    # metrics=[
    #     # tf.keras.metrics.MeanSquaredError(),
    #     # tf.keras.metrics.MeanAbsoluteError(),
    # ],
    )

EPOCHS = 500

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    verbose=1,
    patience=10,
    mode='min',
    restore_best_weights=True)

baseline_history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks = [early_stopping],
)

开始模型初始化 & 训练...
Epoch 1/500


2024-05-30 05:06:46.507021: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x14b75c1d5880 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-05-30 05:06:46.507173: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-05-30 05:06:46.546793: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-30 05:06:46.696585: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


50/50 [==============================] - 15s 191ms/step - loss: 0.4610 - val_loss: 0.0761
Epoch 2/500
50/50 [==============================] - 5s 91ms/step - loss: 0.2794 - val_loss: 0.0466
Epoch 3/500
50/50 [==============================] - 5s 97ms/step - loss: 0.2065 - val_loss: 0.0371
Epoch 4/500
50/50 [==============================] - 5s 94ms/step - loss: 0.1626 - val_loss: 0.0285
Epoch 5/500
50/50 [==============================] - 5s 93ms/step - loss: 0.1266 - val_loss: 0.0236
Epoch 6/500
50/50 [==============================] - 4s 90ms/step - loss: 0.0990 - val_loss: 0.0201
Epoch 7/500
50/50 [==============================] - 5s 94ms/step - loss: 0.0816 - val_loss: 0.0181
Epoch 8/500
11/50 [=====>........................] - ETA: 3s - loss: 0.0697

50/50 [==============================] - 5s 109ms/step - loss: 0.0707 - val_loss: 0.0168
Epoch 9/500
50/50 [==============================] - 4s 88ms/step - loss: 0.0611 - val_loss: 0.0160
Epoch 10/500
50/50 [==============================] - 5s 93ms/step - loss: 0.0535 - val_loss: 0.0154
Epoch 11/500
50/50 [==============================] - 4s 84ms/step - loss: 0.0484 - val_loss: 0.0150
Epoch 12/500
50/50 [==============================] - 5s 90ms/step - loss: 0.0424 - val_loss: 0.0146
Epoch 13/500
50/50 [==============================] - 4s 81ms/step - loss: 0.0384 - val_loss: 0.0143
Epoch 14/500
50/50 [==============================] - 4s 83ms/step - loss: 0.0348 - val_loss: 0.0140
Epoch 15/500
50/50 [==============================] - 4s 85ms/step - loss: 0.0322 - val_loss: 0.0138
Epoch 16/500
50/50 [==============================] - 5s 103ms/step - loss: 0.0298 - val_loss: 0.0135
Epoch 17/500
50/50 [==============================] - 5s 91ms/step - loss: 0.0282 - val_loss: 0.0133
Ep

In [14]:
baseline_history.history.keys()

dict_keys(['loss', 'val_loss'])

In [15]:
# plot_metrics(baseline_history)

In [16]:
# model.save(f'./tf_models/{benchmark}/')

In [17]:
# 输出回测预测
print("开始保存回测预测结果...")
model_pred_result = model.predict(test_ds)
output_df = test_data[['code', 'code_name', 'datetime']]
output_df['label'] = test_df['label']
output_df['label_pred'] = model_pred_result
output_df = output_df.rename(columns={
    'code': 'stock_code',
    'code_name': 'stock_name'
})
output_df.to_pickle(f'../../Offline/backtest/backtest_data/test/{benchmark}_{test_start_date}_回归任务_v6.pkl') 

开始保存回测预测结果...
59/59 [==============================] - 7s 108ms/step


In [18]:
# output_df.head()

In [4]:
import pandas as pd
df = pd.read_pickle('../../Offline/backtest/backtest_data/test/000016_2019-01-01_回归任务_v5.pkl')

In [5]:
def get_model_prediction(model_prediction):
    def get_stock_for_buy(group):
        # group = group[group["label_pred"] > self.params.buy_pred_upper_bound]  # 使用模型预测打分的分布（> 0.9的quantile）进行买入过滤
        select_n = group.nlargest(3, "label_pred")
        return select_n.to_dict("records")

    def get_stock_for_sell(group):
        # group = group[group["label_pred"] < self.params.sell_pred_lower_bound]  # 使用模型预测打分的分布（< 0.1的quantile）进行卖出过滤
        select_n = group.nsmallest(3, "label_pred")
        return select_n.to_dict("records")

    stock_for_buy = model_prediction.groupby("datetime").apply(get_stock_for_buy).to_dict()
    stock_for_sell = model_prediction.groupby("datetime").apply(get_stock_for_sell).to_dict()
    return stock_for_buy, stock_for_sell

In [6]:
stock_for_buy, stock_for_sell = get_model_prediction(df)

In [9]:
# stock_for_buy['2019-01-02']